## Importing necessary modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip uninstall -y numpy pandas
!pip install numpy==1.26.4 pandas==2.2.2

# imgaug (compatible with older NumPy)
!pip install -q imgaug==0.4.0

# Downgrade NumPy to avoid issues with imgaug (NumPy ≥ 2.0 breaks imgaug)
!pip install -q numpy==1.26.4

!pip install tensorflow==2.15.0

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (19 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
Using cached pandas-2.2.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.75 requires ml_dtypes>=0.5.0, but you have ml-dtypes 0.2.0 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
jax 0.5.2 requires m

In [ ]:
import time
from tqdm import tqdm # Cool progress bar

import random
import numpy as np
import pandas as pd
import sys
import os # read and manipulate local files

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cv2
import seaborn as sns

from PIL import Image

import tensorflow.keras as ks
import tensorflow as tf

from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K # F1-score metric

from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import   Lambda, Conv2D, MaxPool2D, UpSampling2D, BatchNormalization, Flatten
from tensorflow.keras.layers import  GlobalAveragePooling2D, Reshape, Multiply, Attention, add,Resizing,  Input, Dense
from tensorflow.keras.layers import Activation,AveragePooling2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate
from tensorflow.keras.models import Model, Sequential

# hide wornings
import warnings
warnings.filterwarnings('ignore')

# from keras import ops# For deeplab3

# import keras_cv # basnet

from imgaug import augmenters as iaa ## for augmentation

In [ ]:
!pip install tensorflow-addons
!pip install validators

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 1.9 MB/s eta 0:00:00


## Defining paths

In [ ]:
model_name = 'TransUNet_2500'

################################################################################################
## SETTING THE PATHS
PATH_TO_CODE ='/content/drive/MyDrive/TRIAL_v1/segmentation_task'
# PATH_TO_CODE =''
sys.path.append(PATH_TO_CODE)

# DATASET_PATH = r'C:\Users\ASUS\Desktop\segmentation & cropping\segmentation_task\segmentation_data43_resized_cropped_split'
DATASET_PATH = r'/content/drive/MyDrive/TRIAL_v1/segmentation_task/segmentation_data43_resized_cropped_split'
# DATASET_PATH = r'segmentation_data43_resized_cropped_split'
DATASET_PATH_TRAIN = os.path.join(DATASET_PATH, 'train')
DATASET_PATH_TEST = os.path.join(DATASET_PATH, 'test')


PATH_TO_SAVE_RESULT = os.path.join(PATH_TO_CODE, 'saved_outputs_segmentation_AUC', model_name)
PATH_BEST_SAVE_WEIGHT = os.path.join(PATH_TO_SAVE_RESULT,'saved_weights')
PATH_TO_SAVE_MODEL = os.path.join(PATH_TO_SAVE_RESULT, 'saved_models')
PATH_SAVE_HISTORY = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_history.csv')
PATH_SAVE_TIME = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_training_time.csv')
PATH_SAVE_AUGMENT_SAMPLE = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_augmented_sample.pdf')
PATH_SAVE_PIXEL_PERCENTAGE_PLOT = os.path.join(PATH_TO_SAVE_RESULT, model_name+'_pixel_percentage.pdf')
# PATH_TO_SAVE_TUNER = os.path.join(PATH_TO_RESULT, 'saved_tuner_model')
################################################################################################

if not os.path.exists(PATH_TO_SAVE_RESULT):
 # If it does not exist, create it
    os.makedirs(PATH_TO_SAVE_RESULT)
################################################################################################
IMG_HEIGHT = 256
IMG_WIDTH = 256
ORIGINAL_IMAGE_SIZE = (IMG_HEIGHT, IMG_WIDTH)
COLOR_CHANNEL = 3

RESIZE_SHAPE = ORIGINAL_IMAGE_SIZE #(128, 128) #ORIGINAL_IMAGE_SIZE#(128, 128)# # #ORIGINAL_IMAGE_SIZE# (128, 128)#
MODEL_INPUT_SIZE = (RESIZE_SHAPE[0], RESIZE_SHAPE[1], COLOR_CHANNEL)

VALIDATION_SPLIT= 0.2
NUM_CLASSES = 2 # Disease and not disease

BATCH_SIZE = 10 # originally was 20
EPOCHS = 1#66

TOTAL_DATA = 2500
################################################################################################

################################################################################################

SAVE_RESULTS = True
SHOW_RESULTS = True

################################################################################################
# Setting the seed
SEED  = 1234
RNG = np.random.default_rng(SEED) # Random number generator
tf.random.set_seed(SEED)

################################################################################################
# Checkpoint parameters val_binary_io_u
SCORE_TO_MONITOR = 'val_binary_io_u' # Score that checkpoints monitor during training
SCORE_OBJECTIVE  = 'max'          # 'max' or 'min', specifies whether the objective is to maximize the score or minimize it.
PATIENCE_LR_REDUCE = 3
MIN_LR = 1e-8
REDUCTION_FACTOR = 0.5            # Factor which lr will be reduced with at plateau
# COOLDOWN_EPOCHS  = 2 #cooldown: Integer. Number of epochs to wait before resuming normal operation after the learning rate has been reduced.

INITIAL_LR = 0.0001

## Defining performance metrics

Official segmentation metrics by keras: https://ks.io/api/metrics/segmentation_metrics/

In [ ]:
def f1_score(y_true, y_pred): # Dice coefficient
    """
    Calculate the F1 score, the harmonic mean of precision and recall, for binary classification.

    Args:
        y_true (Tensor): True binary labels.
        y_pred (Tensor): Predicted probabilities.

    Returns:
        float32: F1 score as a scalar.
    """
    # True Positives: round product of y_true and y_pred
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    # Actual Positives: round y_true
    P = K.sum(K.round(K.clip(y_true, 0, 1)))
    # Recall: TP / Actual Positives
    recall = TP / (P + K.epsilon())

    # Predicted Positives: round y_pred
    Pred_P = K.sum(K.round(K.clip(y_pred, 0, 1)))
    # Precision: TP / Predicted Positives
    precision = TP / (Pred_P + K.epsilon())

    # F1 Score: harmonic mean of precision and recall
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

# source: https://www.tensorflow.org/api_docs/python/tf/keras/metrics/BinaryIoU
iou_score_binary = tf.keras.metrics.BinaryIoU(target_class_ids=[0, 1], threshold=0.5)

def accuracy_score(y_true, y_pred):
    """
    Calculate accuracy score between two binary masks.
    """
    correct = np.sum(y_true == y_pred)  # Count correct predictions
    total = y_true.size  # Total number of pixels
    return correct / total  # Accuracy calculation

def precision_score(groundtruth_mask, pred_mask):
    """
    Calculate precision score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_pred = np.sum(pred_mask)  # Sum of predicted positives
    return np.mean(intersect / total_pixel_pred)  # Precision calculation

def recall_score(groundtruth_mask, pred_mask):
    """
    Calculate recall score between two binary masks.
    """
    intersect = np.sum(pred_mask * groundtruth_mask)  # Calculate intersection
    total_pixel_truth = np.sum(groundtruth_mask)  # Sum of actual positives
    return np.mean(intersect / total_pixel_truth)  # Recall calculation

## Reading the data

In [ ]:
# Function to read images from a directory using Pillow
def read_images_from_directory(directory_path):
    images = []
    for filename in sorted(os.listdir(directory_path)):
        # filename= filename.lower()

        if filename.endswith(('.png', '.jpg', '.JPG' )):  # Check for image file extensions
            img_path = os.path.join(directory_path, filename)
            img = Image.open(img_path)
            img_array = np.array(img)  # Convert the image to a numpy array if needed
            if img_array is not None:
                images.append(img_array)
    return images

# Paths to masks and original images within the dataset
masks_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_masks_binarised')
originals_path_train = os.path.join(DATASET_PATH_TRAIN, 'data43_original')

# Read images into variables
x_train = read_images_from_directory(originals_path_train)  # Original images
y_train = read_images_from_directory(masks_path_train)  # Masks

# Paths to masks and original images within the test dataset
masks_path_test = os.path.join(DATASET_PATH_TEST, 'data43_masks_binarised')
originals_path_test = os.path.join(DATASET_PATH_TEST, 'data43_original')

# Read images into variables
x_test = read_images_from_directory(originals_path_test)  # Original images
y_test = read_images_from_directory(masks_path_test)  # Masks

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 616 original images into x_train
Loaded 616 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Augmentations

In [ ]:
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of the images
    iaa.Flipud(0.2),  # vertically flip 20% of the images
    iaa.Affine(
        scale={"x": (0.8, 1.1), "y": (0.8, 1.1)},  # zoom in or out (80-120%)
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},  # width & height shift (-20 to +20%)
        rotate=(-15, 15),  # rotation (-45 to 45 degrees)
        shear=(-16, 16)  # shear (-16 to 16 degrees)
    ),
    iaa.ElasticTransformation(alpha=(0, 2.0), sigma=0.25),  # apply elastic deformations
    iaa.GaussianBlur(sigma=(0, 2.0))  # apply Gaussian blur
])

In [ ]:
# Function to augment a batch of images and masks
def augment_batch(images, masks, seq, batch_size):
    augmented_images = []
    augmented_masks = []
    while len(augmented_images) < batch_size:
        aug_images, aug_masks = seq(images=images, segmentation_maps=np.expand_dims(masks, axis=-1))
        augmented_images.extend(aug_images)
        augmented_masks.extend(aug_masks)

        if len(augmented_images) >= batch_size:
            break

    # Ensure we only take as many as we need to reach the desired batch_size
    augmented_images = augmented_images[:batch_size]
    augmented_masks = augmented_masks[:batch_size]

    return np.array(augmented_images), np.squeeze(np.array(augmented_masks), axis=-1)

In [ ]:
num_augmented_images_needed = TOTAL_DATA  - len(x_train)  # Calculate how many augmented images we need
if num_augmented_images_needed > 0:
    augmented_x, augmented_y = augment_batch(x_train, y_train, seq, num_augmented_images_needed)
    # Concatenate the original and augmented datasets
    combined_x_train = np.concatenate((x_train, augmented_x), axis=0)
    combined_y_train = np.concatenate((y_train, augmented_y), axis=0)
else:
    combined_x_train = x_train
    combined_y_train = y_train

In [ ]:
x_train = combined_x_train
y_train = combined_y_train

## Checking if the masks are correct

In [ ]:
def has_other_values(images):
    """
    Check if the numpy array 'images' contains any pixel values other than 0 and 255.

    Parameters:
    images (list of numpy.ndarray): A list of numpy arrays, each containing image data.

    Returns:
    str: A message indicating whether other pixel values are present.
    """
    # Iterate through each image in the list
    for image in images:
        # Check if any value in the image is not equal to 0 or 255
        if np.any((image != 0) & (image != 255)):
            return "Has other pixel values"
    return "Only contains 0 and 255"

# Example usage with y_train and y_test (assumed to be lists of numpy arrays)

# Check y_train for any values other than 0 and 255
result_train = has_other_values(y_train)
print("y_train:", result_train)

# Check y_test for any values other than 0 and 255
result_test = has_other_values(y_test)
print("y_test:", result_test)

y_train: Only contains 0 and 255
y_test: Only contains 0 and 255


## Preprocessing DO NOT RESIZE THE IMAGE-- IT GIVES FINE LINES IN THE OUTPUT

In [ ]:
# # # Resizing
# x_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_train]
# y_train = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_train]

# x_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in x_test]
# y_test = [np.array(Image.fromarray(image).resize(RESIZE_SHAPE)) for image in y_test]

# Convert the list to a NumPy array
x_train = np.array(x_train)
y_train = np.array(y_train)

x_test = np.array(x_test)
y_test = np.array(y_test)

# Expand mask dimensions to (batch, 256, 256, 1)
y_train = np.expand_dims(y_train, axis=-1)
y_test = np.expand_dims(y_test, axis=-1)

# Normalizing input between [0,1]
x_train = x_train.astype("float32")/ np.max(x_train)
x_test  = x_test.astype("float32")/np.max(x_test)

y_train = y_train.astype("float32")/ np.max(y_train)
y_test  = y_test.astype("float32")/np.max(y_test)

# Now x_train contains original images, and y_train contains mask images
print(f"Loaded {len(x_train)} original images into x_train")
print(f"Loaded {len(y_train)} mask images into y_train")

# Similarly, for test images
print(f"Loaded {len(x_test)} original images into x_test")
print(f"Loaded {len(y_test)} mask images into y_test")

Loaded 2500 original images into x_train
Loaded 2500 mask images into y_train
Loaded 23 original images into x_test
Loaded 23 mask images into y_test


## Checking if the masks are correct

In [ ]:
def has_other_values(images):
    """
    Check if the numpy array 'images' contains any pixel values other than 0 and 1.

    Parameters:
    images (list of numpy.ndarray): A list of numpy arrays, each containing image data.

    Returns:
    str: A message indicating whether other pixel values are present.
    """
    # Iterate through each image in the list
    for image in images:
        # Check if any value in the image is not equal to 0 or 1
        if np.any((image != 0) & (image != 1)):
            return "Has other pixel values"
    return "Only contains 0 and 1"

# Example usage with y_train and y_test (assumed to be lists of numpy arrays)

# Check y_train for any values other than 0 and 1
result_train = has_other_values(y_train)
print("y_train:", result_train)

# Check y_test for any values other than 0 and 1
result_test = has_other_values(y_test)
print("y_test:", result_test)

y_train: Only contains 0 and 1
y_test: Only contains 0 and 1


## Essential functions

In [ ]:
def fit_and_save_best_model(model_name, model, epochs=EPOCHS):
    ##########################################################################################################
    saved_best_model_name = 'best_'+model_name+'.h5'

    model_saving_path = os.path.join(PATH_TO_SAVE_MODEL, saved_best_model_name)
    # Set up a model checkpoint to save the best model during training
    best_model_callback= ModelCheckpoint(model_saving_path,
                                          monitor=SCORE_TO_MONITOR,
                                          save_best_only=True,
                                          mode=SCORE_OBJECTIVE,
                                          verbose=1)

    EarlyStop_callback = EarlyStopping(min_delta=0.001, patience=10, restore_best_weights=True)

    reduce_LR = ReduceLROnPlateau(
        factor=REDUCTION_FACTOR,      # Factor by which the learning rate will be reduced. new_lr = lr * factor
        patience=PATIENCE_LR_REDUCE, # original was 5      # Number of epochs with no improvement after which learning rate will be reduced.
        verbose=1,       # int. 0: quiet, 1: update messages.
        min_lr=MIN_LR   # Lower bound on the learning rate.
    )

    my_callbacks = [best_model_callback , reduce_LR, EarlyStop_callback]
    ##########################################################################################################

    start_time = time.time()

    # Fitting the model
    train_history = model.fit(
        x_train,
        y_train,
        epochs= epochs,
        batch_size=BATCH_SIZE,
        validation_split= VALIDATION_SPLIT,
        callbacks=my_callbacks,
    )
    total_time = time.time() - start_time

    return saved_best_model_name, train_history, total_time

## Models
### Models worked
- Model01: Basic basic_unet
- Model02: unet_vgg16 (my own custom)
- Model03: TransUNet
- Model04: DeepLab-v3Plus
- Model05: ResUNet
- Model06: UNet_with_attention
- Model07: Multi-resUnet
- Model08: Inception_resnetV2 (my own custom)



### TransUNet--works with batch20-augmented data (could not check with 30--memory issue)
- `image_size`=224
- `patch_size`=16
- `hybrid`=True
- `grid`=(14,14)
- `hidden_size`=768
- `n_layers`=12
- `n_heads`=12
- `mlp_dim`=3072
- `dropout`=0.1
- `decoder_channels`=[256,128,64,16]
- `n_skip`=3
- `num_classes`=3
- `final_act`='sigmoid'
- `pretrain`=True
- `freeze_enc_cnn`=True

In [ ]:
! pip install tensorflow_addons
! pip install validators

In [ ]:
## Install: pip install validators
from transunet import TransUNet

def get_TransUNet(n_filters=16, batchnorm=True):
    """--- COde runs--run it in PC2
       # Source: https://github.com/awsaf49/TransUNet-tf/tree/main
    """
    model = TransUNet(image_size=MODEL_INPUT_SIZE[0],
                      pretrain=True,
                     num_classes=1)

    optimizer = Adam(learning_rate=INITIAL_LR)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', f1_score, iou_score_binary],
                  optimizer=optimizer)

    return model

In [ ]:
import tensorflow_addons as tfa

tfk = tf.keras
tfkl = tfk.layers
tfm = tf.math
L2_WEIGHT_DECAY = 1e-4
## Extra custom layer for Reloading the best model
class AddPositionEmbs(tfkl.Layer):
    """Adds (optionally learned) positional embeddings to the inputs."""

    def __init__(self, trainable=True, **kwargs):
        super().__init__(trainable=trainable, **kwargs)
        self.trainable = trainable

    def build(self, input_shape):
        assert (
            len(input_shape) == 3
        ), f"Number of dimensions should be 3, got {len(input_shape)}"
        self.pe = tf.Variable(
            name="pos_embedding",
            initial_value=tf.random_normal_initializer(stddev=0.06)(
                shape=(1, input_shape[1], input_shape[2])
            ),
            dtype="float32",
            trainable=self.trainable,
        )

    def call(self, inputs):
        return inputs + tf.cast(self.pe, dtype=inputs.dtype)

class TransformerBlock(tfkl.Layer):
    """Implements a Transformer block."""

    def __init__(self, *args, n_heads, mlp_dim, dropout, trainable=True, **kwargs):
        super().__init__(*args, trainable=trainable, **kwargs)
        self.n_heads = n_heads
        self.mlp_dim = mlp_dim
        self.dropout = dropout

    def build(self, input_shape):
        self.att = MultiHeadSelfAttention(
            n_heads=self.n_heads,
            name="MultiHeadDotProductAttention_1",
        )
        self.mlpblock = tfk.Sequential(
            [
                tfkl.Dense(
                    self.mlp_dim,
                    activation="linear",
                    name=f"{self.name}/Dense_0"
                ),
                tfkl.Lambda(
                    lambda x: tfk.activations.gelu(x, approximate=False)
                )
                if hasattr(tfk.activations, "gelu")
                else tfkl.Lambda(
                    lambda x: tfa.activations.gelu(x, approximate=False)
                ),
                tfkl.Dropout(self.dropout),
                tfkl.Dense(
                    input_shape[-1], name=f"{self.name}/Dense_1"),
                tfkl.Dropout(self.dropout),
            ],
            name="MlpBlock_3",
        )
        self.layernorm1 = tfkl.LayerNormalization(
            epsilon=1e-6, name="LayerNorm_0"
        )
        self.layernorm2 = tfkl.LayerNormalization(
            epsilon=1e-6, name="LayerNorm_2"
        )
        self.dropout = tfkl.Dropout(self.dropout)

    def call(self, inputs, training):
        x = self.layernorm1(inputs)
        x, weights = self.att(x)
        x = self.dropout(x, training=training)
        x = x + inputs
        y = self.layernorm2(x)
        y = self.mlpblock(y)
        return x + y, weights



class SegmentationHead(tfkl.Layer):
    def __init__(self, name="seg_head",
                 num_classes=9,
                 kernel_size=1,
                 final_act='sigmoid',
                 ** kwargs):
        super(SegmentationHead, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.kernel_size = kernel_size
        self.final_act  = final_act

    def build(self, input_shape):
        self.conv = tfkl.Conv2D(
            filters=self.num_classes, kernel_size=self.kernel_size, padding="same",
            kernel_regularizer=tfk.regularizers.L2(L2_WEIGHT_DECAY),
            kernel_initializer=tfk.initializers.LecunNormal())
        self.act = tfkl.Activation(self.final_act)

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.act(x)
        return x


class Conv2DReLu(tfkl.Layer):
    def __init__(self, filters, kernel_size, padding="same", strides=1, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size
        self.padding = padding
        self.strides = strides

    def build(self, input_shape):
        self.conv = tfkl.Conv2D(
            filters=self.filters, kernel_size=self.kernel_size, strides=self.strides,
            padding=self.padding, use_bias=False, kernel_regularizer=tfk.regularizers.L2(L2_WEIGHT_DECAY),
            kernel_initializer="lecun_normal")

        self.bn = tfkl.BatchNormalization(momentum=0.9, epsilon=1e-5)

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = tf.nn.relu(x)
        return x


class DecoderBlock(tfkl.Layer):
    def __init__(self, filters, **kwargs):
        super().__init__(**kwargs)
        self.filters = filters

    def build(self, input_shape):
        self.conv1 = Conv2DReLu(filters=self.filters, kernel_size=3)
        self.conv2 = Conv2DReLu(filters=self.filters, kernel_size=3)
        self.upsampling = tfkl.UpSampling2D(
            size=2, interpolation="bilinear")

    def call(self, inputs, skip=None):
        x = self.upsampling(inputs)
        if skip is not None:
            x = tf.concat([x, skip], axis=-1)
        x = self.conv1(x)
        x = self.conv2(x)
        return x


class DecoderCup(tfkl.Layer):
    def __init__(self, decoder_channels, n_skip=3, **kwargs):
        super().__init__(**kwargs)
        self.decoder_channels = decoder_channels
        self.n_skip = n_skip

    def build(self, input_shape):
        self.conv_more = Conv2DReLu(filters=512, kernel_size=3)
        self.blocks = [DecoderBlock(filters=out_ch)
                       for out_ch in self.decoder_channels]

    def call(self, hidden_states, features):
        x = self.conv_more(hidden_states)
        for i, decoder_block in enumerate(self.blocks):
            if features is not None:
                skip = features[i] if (i < self.n_skip) else None
            else:
                skip = None
            x = decoder_block(x, skip=skip)
        return x

class MultiHeadSelfAttention(tfkl.Layer):
    def __init__(self, *args, trainable=True, n_heads, **kwargs):
        super().__init__(trainable=trainable, *args, **kwargs)
        self.n_heads = n_heads

    def build(self, input_shape):
        hidden_size = input_shape[-1]
        n_heads = self.n_heads
        if hidden_size % n_heads != 0:
            raise ValueError(
                f"embedding dimension = {hidden_size} should be divisible by number of heads = {n_heads}"
            )
        self.hidden_size = hidden_size
        self.projection_dim = hidden_size // n_heads
        self.query_dense = tfkl.Dense(
            hidden_size, name="query")
        self.key_dense = tfkl.Dense(
            hidden_size, name="key")
        self.value_dense = tfkl.Dense(
            hidden_size, name="value")
        self.combine_heads = tfkl.Dense(
            hidden_size, name="out")

    # pylint: disable=no-self-use
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], score.dtype)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(
            x, (batch_size, -1, self.n_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)
        key = self.key_dense(inputs)
        value = self.value_dense(inputs)
        query = self.separate_heads(query, batch_size)
        key = self.separate_heads(key, batch_size)
        value = self.separate_heads(value, batch_size)

        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.hidden_size))
        output = self.combine_heads(concat_attention)
        return output, weights

## Train and Results

In [ ]:
# model= get_basic_unet() ## Model1
# model= get_unet_vgg16() ## Model2
model= get_TransUNet() ## Model3
# model= get_DeepLabV3Plus() ## Model4
# model = get_ResUNet_v3() ## Model5
# model = UNet_with_attention() ## Model6
# model = get_multiresunet() ## Model7
# model = get_unet_InceptionResNetV2() ## Model8

model.summary()

461217452/461217452 [==============================] - 13s 0us/step
Model: "TransUNet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 130, 130, 64)         0         ['conv1_conv[0][0]']          
                      

## Loading the best model

In [ ]:
#Now, load the best model
# best_model = ks.models.load_model(os.path.join(PATH_TO_SAVE_MODEL, best_model_name),
#                                   custom_objects={'f1_score': f1_score,
#                                                   'binary_io_u':iou_score_binary,
#                                                   'AddPositionEmbs': AddPositionEmbs,
#                                                  'TransformerBlock':TransformerBlock,
#                                                  'DecoderCup':DecoderCup,
#                                                  'SegmentationHead':SegmentationHead,
#                                                  'MultiHeadSelfAttention':MultiHeadSelfAttention,
#                                                  'Conv2DReLu':Conv2DReLu})
# path ='/content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation/TransUNet/saved_models/best_TransUNet.h5'

# #Now, load the best model
# best_model = ks.models.load_model(path,
#                                   custom_objects={'f1_score': f1_score,
#                                                   'binary_io_u':iou_score_binary,
#                                                   'AddPositionEmbs': AddPositionEmbs,
#                                                  'TransformerBlock':TransformerBlock,
#                                                  'DecoderCup':DecoderCup,
#                                                  'SegmentationHead':SegmentationHead,
#                                                  'MultiHeadSelfAttention':MultiHeadSelfAttention,
#                                                  'Conv2DReLu':Conv2DReLu})
# # best_model =model

best_model = ks.models.load_model("/content/drive/MyDrive/TRIAL_v1/segmentation_task/saved_outputs_segmentation/2500/TransUNet_2500_v2_batch10_TAKE/saved_models/best_TransUNet_2500_v2_batch10.h5",
                                  custom_objects={'f1_score': f1_score,
                                                  'binary_io_u':iou_score_binary,
                                                  'AddPositionEmbs': AddPositionEmbs,
                                                 'TransformerBlock':TransformerBlock,
                                                 'DecoderCup':DecoderCup,
                                                 'SegmentationHead':SegmentationHead,
                                                 'MultiHeadSelfAttention':MultiHeadSelfAttention,
                                                 'Conv2DReLu':Conv2DReLu})

## Visualize model predictions

In [ ]:
# Get predictions from the model
predictions = best_model.predict(x_test, verbose=1)

USER_DETERMINED_THRESHOLD = 0.5
thresholded_predictions  = (predictions  >= USER_DETERMINED_THRESHOLD)
# Removing the color channel
thresholded_predictions_without_color_channel = np.squeeze(thresholded_predictions, axis=-1)

1/1 [==============================] - 7s 7s/step


# AUC

In [ ]:
from sklearn.metrics import roc_auc_score

# Flatten masks for AUC computation
y_true_flat = y_test.flatten()
y_score_flat = predictions.flatten()  # Predicted probabilities

In [ ]:
# Bootstrapping AUC
n_iterations = 100
rng = np.random.default_rng(SEED)
bootstrap_aucs = []

for _ in range(n_iterations):
    indices = rng.integers(0, len(y_true_flat), len(y_true_flat))
    y_true_sample = y_true_flat[indices]
    y_score_sample = y_score_flat[indices]

    # Ensure both classes are present in the sample
    if len(np.unique(y_true_sample)) < 2:
        continue

    auc = roc_auc_score(y_true_sample, y_score_sample)
    bootstrap_aucs.append(auc)

In [ ]:
# Calculate statistics
mean_auc = np.mean(bootstrap_aucs)
std_auc = np.std(bootstrap_aucs)
lower_ci = np.percentile(bootstrap_aucs, 2.5)
upper_ci = np.percentile(bootstrap_aucs, 97.5)

# Print AUC statistics
print(f"Bootstrapped AUC: Mean={mean_auc:.4f}, Std={std_auc:.4f}, 95% CI=({lower_ci:.4f}, {upper_ci:.4f})")

# Optional saving
if SAVE_RESULTS:
    auc_df = pd.DataFrame({'AUC Scores': bootstrap_aucs})
    auc_df.to_csv(os.path.join(PATH_TO_SAVE_RESULT, f"{model_name}_bootstrapped_auc.csv"), index=False)

Bootstrapped AUC: Mean=0.9734, Std=0.0005, 95% CI=(0.9726, 0.9744)
